In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dfE = pd.read_csv('events_up_to_01062018.csv')
dfT = pd.read_csv('labels_training_set.csv')

c:\users\ignacio\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#OHE a columna event
dfD = dfE[['person', 'event']]
dfD = pd.get_dummies(dfD, columns=['event']).groupby('person').sum()

In [4]:
#le agrego las features de event a las labels
dfTD = dfT.sort_values('person').merge(dfD, on='person')

In [5]:
#Agrego columna con ultima visita al sitio
temp = dfE.groupby('person')['timestamp'].max().to_frame()
dfTDb = dfTD.merge(temp, on='person')
dfTDb['month'] = dfTDb['timestamp'].str[5:7]
dfTDb['day'] = dfTDb['timestamp'].str[8:10]
dfTDb[['day','month']] = dfTDb[['day','month']].apply(pd.to_numeric)
dfTDb['last_visit'] = dfTDb['month']*30 + dfTDb['day']
dfTDb = dfTDb.drop(['timestamp','month','day'], axis=1)

In [6]:
#Agrego columna con ultima compra
temp = dfE.loc[dfE['event'] == 'conversion'].groupby('person')['timestamp'].max().to_frame()
dfTDc = dfTDb.merge(temp, how='left', on='person')
dfTDc = dfTDc.fillna('000000000000000000')
dfTDc['month'] = dfTDc['timestamp'].str[5:7]
dfTDc['day'] = dfTDc['timestamp'].str[8:10]
dfTDc[['day','month']] = dfTDc[['day','month']].apply(pd.to_numeric)
dfTDc['last_conversion'] = dfTDc['month']*30 + dfTDc['day']
dfTDc = dfTDc.drop(['timestamp','month','day'], axis=1)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

df_y = dfTDc['label']
df_X = dfTDc.drop(['person', 'label'], axis=1)

GBC = GradientBoostingClassifier(random_state=23,learning_rate=0.06,subsample=0.75, min_samples_split=8)
scores = cross_val_score(GBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8545760253915411